## Prerequesites

Modify `target_directory` in the code to match the path of your desired directory before executing the notebook.

In [1]:
import os
import requests

# Define the target directory (change yours)
target_directory = r'C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search'

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search


#### Configure Environment Variables 

In [3]:
# Local Variables Needed

# SharePoint Configuration
SITE_DOMAIN = 'mngenvmcap747548.sharepoint.com'  # Domain of your SharePoint site
SITE_NAME = 'Contoso'                           # Name of your SharePoint site

# Azure Open AI Deployment Configuration
DEPLOYMENT = "foundational-ada"                 # Deployment name for your Azure Open AI service needed for text embeddings
MODEL_NAME = "text-embedding-ada-002"           # Model name for text embeddings in Azure Open AI service

# Azure AI Search Index Configuration
INDEX_NAME = "langchain-vector-demo-custom"     # Name of the index in Azure Cognitive Search

In [5]:
from gbb_ai.sharepoint_data_extractor import SharePointDataExtractor

# Instantiate the SharePointDataExtractor client
# The client handles the complexities of interacting with SharePoint's REST API, providing an easy-to-use interface for data extraction.
client_scrapping = SharePointDataExtractor()

In [6]:
client_scrapping.load_environment_variables_from_env_file()
client_scrapping.msgraph_auth()

2023-12-14 20:20:45,760 - micro - MainProcess - INFO     Successfully loaded environment variables: TENANT_ID, CLIENT_ID, CLIENT_SECRET (sharepoint_data_extractor.py:load_environment_variables_from_env_file:89)
2023-12-14 20:20:46,555 - micro - MainProcess - INFO     New access token retrieved. (sharepoint_data_extractor.py:msgraph_auth:121)


'eyJ0eXAiOiJKV1QiLCJub25jZSI6InJHTThhajBoamtpNWhVeXpfTHE0R05yMW9odVY5Wm42NWxCRm9ZVDJwQWsiLCJhbGciOiJSUzI1NiIsIng1dCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSIsImtpZCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC85NDk1ZDhjOS00ZWJiLTQxMDctYjkwNS1jN2I0NWQxYjdiN2EvIiwiaWF0IjoxNzAyNjA2NTQ2LCJuYmYiOjE3MDI2MDY1NDYsImV4cCI6MTcwMjYxMDQ0NiwiYWlvIjoiRTJWZ1lEZzQvZXBXbzNPVFB4ZG5QWDg2ajdYNEhnQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJkZXYtZ3JhcGgiLCJhcHBpZCI6IjExODU4M2VlLTk0ZWQtNDVkZC04NzBiLTczNzg0MDQ1ZWIzNyIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0Lzk0OTVkOGM5LTRlYmItNDEwNy1iOTA1LWM3YjQ1ZDFiN2I3YS8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6IjRmNjE0Mzc0LTY1ZmEtNDVmYy04MzY5LWNiNjE2YTZmZTA4ZiIsInJoIjoiMC5BYjBBeWRpVmxMdE9CMEc1QmNlMFhSdDdlZ01BQUFBQUFBQUF3QUFBQUFBQUFBRExBQUEuIiwicm9sZXMiOlsiVGVhbXNBY3Rpdml0eS5SZWFkLkFsbCIsIlNoYXJlUG9pbnRUZW5hbnRTZXR0aW5ncy5SZWFkLkFsbCIsIlBlb3BsZS5SZWFkLkFsbCIsIkdyb3VwLlJlYWQuQWxsIiwiU2l0ZXMuUm

In [5]:
site_id = client_scrapping.get_site_id(site_domain=SITE_DOMAIN, site_name=SITE_NAME)
drive_id = client_scrapping.get_drive_id(site_id=site_id)

2023-12-14 20:17:59,624 - micro - MainProcess - INFO     Getting the Site ID... (sharepoint_data_extractor.py:get_site_id:190)
2023-12-14 20:18:00,315 - micro - MainProcess - INFO     Site ID retrieved: mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0 (sharepoint_data_extractor.py:get_site_id:194)
2023-12-14 20:18:01,090 - micro - MainProcess - INFO     Successfully retrieved drive ID: b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm (sharepoint_data_extractor.py:get_drive_id:211)


In [6]:
url  = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/"

url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root/test/test2/test3:/children"

file_name = "test3.docx"

folder_path="/test/test2/test3/"


url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:{folder_path}{file_name}:/content"

#url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{file_name}:/content"

# url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root/children"
file_name = "test.docx"

# url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{file_name}:/content"


In [7]:
url

'https://graph.microsoft.com/v1.0/sites/mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0/drives/b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm/root:/test/test2/test3/test3.docx:/content'

In [5]:
client_scrapping.get_files_in_site(site_id=site_id, drive_id=drive_id,  folder_path="/test/test2/test3/")

NameError: name 'site_id' is not defined

In [7]:
file = client_scrapping.retrieve_sharepoint_files_content(site_domain=SITE_DOMAIN, site_name=SITE_NAME, folder_path="/test/test2/")

2023-12-14 20:20:53,091 - micro - MainProcess - INFO     Getting the Site ID... (sharepoint_data_extractor.py:get_site_id:190)
2023-12-14 20:20:53,724 - micro - MainProcess - INFO     Site ID retrieved: mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0 (sharepoint_data_extractor.py:get_site_id:194)
2023-12-14 20:20:54,326 - micro - MainProcess - INFO     Successfully retrieved drive ID: b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm (sharepoint_data_extractor.py:get_drive_id:211)
2023-12-14 20:20:54,327 - micro - MainProcess - INFO     Making request to Microsoft Graph API (sharepoint_data_extractor.py:get_files_in_site:250)
2023-12-14 20:20:54,997 - micro - MainProcess - INFO     Received response from Microsoft Graph API (sharepoint_data_extractor.py:get_files_in_site:253)
2023-12-14 20:20:57,007 - micro - MainProcess - INFO     Returning highest priority group: Group_critical (azure_search_security_trimming.

In [8]:
file

[{'page_content': 'A large language model (LLM) is a type of language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre-)trained using self-supervised learning and semi-supervised learning.\nAs autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corp

In [25]:
client_scrapping.get_docx_content(site_id, drive_id, file_name)

2023-12-14 18:35:06,807 - micro - MainProcess - INFO     Starting request for file: test.docx (sharepoint_data_extractor.py:get_docx_content:356)
2023-12-14 18:35:08,042 - micro - MainProcess - INFO     Successfully retrieved content for file: test.docx (sharepoint_data_extractor.py:get_docx_content:369)


'A large language model (LLM) is a type of language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre-)trained using self-supervised learning and semi-supervised learning.\nAs autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5]\nNotable e

In [14]:
client_scrapping._make_ms_graph_request(url)

2023-12-14 18:45:24,307 - micro - MainProcess - ERROR    HTTP Error: 400 Client Error: Bad Request for url: https://graph.microsoft.com/v1.0/sites/mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0/drives/b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm/root/test/test2/test3:/children (sharepoint_data_extractor.py:_make_ms_graph_request:157)


HTTPError: 400 Client Error: Bad Request for url: https://graph.microsoft.com/v1.0/sites/mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0/drives/b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm/root/test/test2/test3:/children

In [8]:
# Retrieve .docx file contents from a specified SharePoint site using SharePointDataExtractor
content_files = client_scrapping.retrieve_sharepoint_files_content(site_domain=SITE_DOMAIN, site_name=SITE_NAME, folder_path="/test/test2/", minutes_ago=None,file_formats=["docx"])

2023-12-14 20:11:32,938 - micro - MainProcess - INFO     Getting the Site ID... (sharepoint_data_extractor.py:get_site_id:190)
2023-12-14 20:11:33,492 - micro - MainProcess - INFO     Site ID retrieved: mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0 (sharepoint_data_extractor.py:get_site_id:194)
2023-12-14 20:11:34,097 - micro - MainProcess - INFO     Successfully retrieved drive ID: b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm (sharepoint_data_extractor.py:get_drive_id:211)
2023-12-14 20:11:34,098 - micro - MainProcess - INFO     Making request to Microsoft Graph API (sharepoint_data_extractor.py:get_files_in_site:250)
2023-12-14 20:11:34,970 - micro - MainProcess - INFO     Received response from Microsoft Graph API (sharepoint_data_extractor.py:get_files_in_site:253)
2023-12-14 20:11:36,809 - micro - MainProcess - INFO     Returning highest priority group: Group_critical (azure_search_security_trimming.

In [9]:
content_files

[{'page_content': 'A large language model (LLM) is a type of language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre-)trained using self-supervised learning and semi-supervised learning.\nAs autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corp

### 2. Chunking, Text Vectorization, and Indexing with `TextChunkingIndexing`

The `TextChunkingIndexing` simplifies the role in chunking, text vectorization, and indexing in Azure AI Search acting as Vector Database. It utilizes Langchain as an orchestrator to simplify and enhance the text proccesing strategy. More about Ai search and LangChain integration [here](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/azure-cognitive-search-and-langchain-a-seamless-integration-for/ba-p/3901448)

#### Key Features of `TextChunkingIndexing`:

- **Text Chunking**: Breaks down extensive text data into smaller chunks based on character count, facilitating easier analysis and indexing.
- **Customization**: Allows for the adjustment of chunk size and overlap, catering to various text processing needs.
- **Text Vectorization**: Transforms the chunked text into vector representations, essential for efficient indexing and retrieval.
- **Indexing to Vector Store**: The vectorized text is then indexed into Azure AI Search, a powerful vector database for storing and retrieving text data.

#### Importance of Chunking Fine-tuning and overlapping:

- Fine-tuning chunk sizes and overlaps is critical for optimizing text retrieval quality, particularly in applications requiring precise search functionalities (relevance), like RAGs. More about fine tuning and aunderatand releveance scores [here](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/azure-cognitive-search-outperforming-vector-search-with-hybrid/ba-p/3929167)

In [6]:
from gbb_ai.langchain_indexing import TextChunkingIndexing

client_indexing = TextChunkingIndexing()

In [7]:
# Instantiate the TextChunkingIndexing client
# This cleint is resposnsinle for chunking text into smaller pieces using Langchaing framework, which are then indexed by Azure AI Search.
client_indexing = TextChunkingIndexing()

client_indexing.setup_aoai()

client_indexing.load_embedding_model(deployment=DEPLOYMENT,model_name=MODEL_NAME)


2023-12-10 12:39:53,214 - micro - MainProcess - INFO     Loading OpenAIEmbeddings object with model text-embedding-ada-002, deployment foundational-ada, and chunk size 1000 (langchain_indexing.py:load_embedding_model:102)
2023-12-10 12:39:53,232 - micro - MainProcess - INFO     OpenAIEmbeddings object created successfully. (langchain_indexing.py:load_embedding_model:115)


OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, async_client=None, model='text-embedding-ada-002', deployment='foundational-ada', openai_api_version='2023-05-15', openai_api_base='https://ml-workspace-dev-eastus-001-aoai.openai.azure.com/', openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key='d050ad8b96ef4ecbb5099eece1212a91', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=16, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=True, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

#### Quick Guide to Setting Up Azure Search Index (Optional)

Let's set up an Azure Search index tailored for advanced search capabilities, including semantic and vector-based searches. Here's a step-by-step guide:

##### Embedding Function Setup:

Define embedding_function to transform text into vectors. This powers the semantic search.

##### Define Index Fields:

Create fields like id, content, content_vector, and others in the fields list. Each field represents a document attribute.
Make sure content_vector aligns with your embedding function's output.

##### Initialize Azure Search Client:

Instantiate AzureSearch with your Azure endpoint, key, custom index_name, and the fields list.
Configure semantic settings to fine-tune search relevance.

##### Customize As Needed:

Modify fields based on your document attributes.
Adjust index_name or semantic configurations to fit your specific search needs.

```python 
from azure.search.documents.indexes.models import (
    SearchFieldDataType, SimpleField, SearchableField, SemanticSettings, SemanticConfiguration, PrioritizedFields, SemanticField
)
from azure.search.documents.models import Vector
from langchain.vectorstores.azuresearch import AzureSearch
from your_embedding_module import embeddings  # Replace with your actual module

# Embedding function and fields setup
embedding_function = embeddings.embed_query
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    # ... other fields ...
]

# Azure Search client initialization
vector_store = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_ADMIN_KEY"),
    index_name="your-custom-index-name",
    embedding_function=embedding_function,
    fields=fields,
    # Semantic settings
    semantic_settings=SemanticSettings(
        default_configuration="config",
        configurations=[
            SemanticConfiguration(
                name="config",
                prioritized_fields=PrioritizedFields(
                    title_field=SemanticField(field_name="content"),
                    # ... other configurations ...
                ),
            )
        ],
    ),
)

# Now, your Azure Search index is ready for advanced querying!
```

In [8]:
# this is the function that will load or create the index mentioned above
client_indexing.setup_azure_search(index_name="langchain-vector-demo-custom")

c:\Users\pablosal\AppData\Local\anaconda3\envs\sharepoint-indexing\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 15.91it/s]
2023-12-10 12:40:00,377 - micro - MainProcess - INFO     Azure Cognitive Search client configured successfully. (langchain_indexing.py:setup_azure_search:202)


In [9]:
# function to split the text from content_files (a list of Document objects) into chunks of 1000 characters and overlap of 200 characters
chuncks = client_indexing.split_documents_in_chunks(content_files, chunk_size=1000, chunk_overlap=200)

In [10]:
# Indexing chunks in Azure AI Search
client_indexing.embed_and_index(texts=chuncks)

100%|██████████| 1/1 [00:00<00:00, 15.74it/s]


### 3. Search with security treamming 

In [1]:
import os
from dotenv import load_dotenv

# Define the target directory
target_directory = r'C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search'

# Load .env file
load_dotenv()

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search


In [2]:
INDEX_NAME = "langchain-vector-demo-custom" 

In [3]:
from gbb_ai.trimming_ai_search import AzureSearchManager

client_search = AzureSearchManager(index_name=INDEX_NAME)

In [4]:
user_groups = client_search.get_current_user_groups()
user_groups

['admins']

In [5]:
search_query = "LLM in International Business Law"

In [6]:
security_group="Group_criticaldfvdm;mvs"
security_group_list=["Group_critical","Group_high","Group_medium","Group_low"]

In [7]:
results = client_search.secure_hybrid_search_rerank(search_query=search_query, security_group=security_group, top_k=5, azure_deployment_name="foundational-ada", semantic_configuration_name="config")

c:\Users\pablosal\AppData\Local\anaconda3\envs\sharepoint-indexing\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00,  1.74it/s]
2023-12-11 17:13:17,458 - micro - MainProcess - INFO     Search Results:
Result 1:
Score: 0.026480834931135178
Reranker Score: 2.258655071258545
Content: A large language model (LLM) is a type of language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre-)trained using self-supervised learning and semi-supervised learning. As autoregressive languag

In [18]:
results

['A large language model (LLM) is a type of language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre-)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5]',
 "Flamin

In [ ]:
if not self.token:
    logger.error("Access token is not available.")
    return []

headers = {
    'Authorization': f'Bearer {self.token}',
    'Content-Type': 'application/json'
}
endpoint = f"https://graph.microsoft.com/v1.0/users/{user_id}/memberOf"

try:
    response = requests.get(endpoint, headers=headers)
    if response.status_code != 200:
        logger.error(f"Error retrieving user groups: {response.status_code} {response.reason}")
        logger.error(f"Response content: {response.text}")
        return []
    return response.json()
except Exception as e:
    logger.error(f"Exception in retrieving user groups: {e}")
    return []